In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

In [4]:
ratings = pd.read_csv("drive/MyDrive/Dataset2/ratings.csv")
movies = pd.read_csv("drive/MyDrive/Dataset2/movies.csv")

In [5]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [6]:
ratings = ratings[['userId', 'movieId', 'rating']]
ratings.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [7]:
movies.shape

(9742, 3)

In [8]:
ratings.shape

(100836, 3)

In [9]:
x=ratings['userId'].value_counts()>200

In [14]:
y = x[x].index

In [19]:
y

Index([414, 599, 474, 448, 274, 610,  68, 380, 606, 288,
       ...
       119, 563, 263,  73, 586, 220, 246, 234, 452, 385],
      dtype='int64', name='userId', length=133)

In [20]:
ratings = ratings[ratings['userId'].isin(y)]

In [22]:
ratings.shape

(68284, 3)

In [23]:
ratings.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [25]:
movies_ratings = pd.merge(ratings, movies, on="movieId")
movies_ratings.head()

,userId,movieId,rating,title,genres
0,1,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,18,1,3.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,19,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,21,1,3.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,45,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [26]:
movies_ratings.shape

(68284, 5)

In [27]:
numbrer_rating = movies_ratings.groupby('title')['rating'].count().reset_index()

In [28]:
numbrer_rating.rename(columns={'rating':'No.Of Rating'}, inplace=True)

In [29]:
numbrer_rating.head()

,title,No.Of Rating
0,'71 (2014),1
1,'Hellboy': The Seeds of Creation (2004),1
2,'Round Midnight (1986),1
3,'Tis the Season for Love (2015),1
4,"'burbs, The (1989)",12


In [30]:
final_rating = movies_ratings.merge(numbrer_rating, on='title')

In [31]:
final_rating.head()

,userId,movieId,rating,title,genres,No.Of Rating
0,1,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,92
1,18,1,3.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,92
2,19,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,92
3,21,1,3.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,92
4,45,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,92


In [32]:
final_rating.shape

(68284, 6)

In [33]:
final_rating = final_rating[final_rating['No.Of Rating'] >= 50]

In [34]:
final_rating.shape

(13839, 6)

In [35]:
movie_pivot = final_rating.pivot_table(columns='userId', index='title', values='rating')

In [38]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
movie_pivot.head()


userId,1,4,6,18,19,20,21,28,41,42,45,50,51,57,62,63,64,66,68,73,82,84,89,91,103,104,105,111,119,122,125,132,135,140,156,160,169,177,182,186,187,198,199,200,202,212,217,219,220,221,222,226,232,234,239,246,249,263,274,275,282,288,290,292,294,298,304,305,307,312,313,318,325,328,330,332,339,352,354,356,357,365,368,372,380,381,382,385,387,391,414,425,428,432,434,438,448,452,453,462,469,474,477,480,483,484,489,495,509,514,517,525,534,555,560,561,562,563,567,573,580,586,590,594,596,597,599,600,603,605,606,608,610
title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2001: A Space Odyssey (1968),NaN,NaN,NaN,4.0,3.0,NaN,NaN,3.0,2.0,5.0,4.0,4.5,5.0,5.0,NaN,3.5,1.0,NaN,1.5,4.0,NaN,NaN,NaN,3.5,4.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,4.5,5.0,NaN,1.5,5.0,5.0,4.0,4.0,NaN,NaN,5.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,4.5,NaN,2.5,1.0,NaN,5.0,5.0,NaN,NaN,4.0,4.0,4.0,3.5,5.0,3.0,3.5,5.0,4.0,1.5,NaN,5.0,4.0,3.5,NaN,NaN,NaN,3.0,2.0,4.0,NaN,NaN,4.0,5.0,5.0,5.0,NaN,NaN,NaN,4.5,NaN,4.0,3.0,NaN,2.0,5.0,4.0,4.5,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,1.0,NaN,3.5,NaN,5.0,NaN,4.0,NaN,5.0,4.0,5.0,NaN,5.0,3.0,4.5
300 (2007),NaN,NaN,NaN,3.5,NaN,NaN,3.5,4.5,2.0,NaN,NaN,NaN,5.0,NaN,4.5,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.5,4.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,2.5,NaN,NaN,5.0,NaN,3.5,NaN,4.0,4.5,NaN,NaN,NaN,5.0,NaN,4.0,NaN,5.0,NaN,4.0,NaN,3.5,NaN,NaN,NaN,NaN,3.0,NaN,3.0,3.5,NaN,NaN,3.5,NaN,NaN,NaN,NaN,NaN,4.0,3.5,3.0,NaN,3.5,NaN,NaN,4.0,3.5,NaN,NaN,3.5,NaN,4.0,NaN,NaN,4.0,3.5,5.0,3.0,NaN,NaN,NaN,NaN,NaN,4.5,4.0,4.0,3.0,4.0,NaN,3.5,NaN,NaN,3.5,4.0,NaN,3.5,3.5,NaN,NaN,NaN,4.5,5.0,NaN,3.5,NaN,NaN,NaN,3.0,NaN,NaN,3.0,NaN,5.0,4.0
Ace Ventura: Pet Detective (1994),NaN,NaN,3.0,2.5,2.0,NaN,NaN,NaN,NaN,2.0,4.0,NaN,3.0,NaN,3.0,5.0,3.5,NaN,2.5,NaN,3.5,NaN,NaN,3.0,3.5,NaN,NaN,NaN,NaN,NaN,NaN,1.5,NaN,NaN,3.0,NaN,3.5,4.0,NaN,NaN,NaN,4.0,NaN,4.0,NaN,NaN,3.0,4.0,2.0,NaN,4.0,4.5,NaN,2.0,NaN,NaN,3.5,NaN,4.5,NaN,2.0,3.0,NaN,2.0,NaN,3.0,NaN,NaN,4.0,NaN,NaN,3.5,NaN,NaN,0.5,NaN,NaN,NaN,2.5,1.5,NaN,NaN,NaN,NaN,5.0,3.0,NaN,3.0,3.5,NaN,2.0,3.5,0.5,1.5,3.0,4.0,3.0,NaN,4.0,NaN,NaN,1.0,3.0,3.5,4.0,NaN,1.5,4.5,4.0,NaN,2.5,NaN,3.5,4.0,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,2.5,3.0,NaN,NaN,NaN,3.5,3.0
Airplane! (1980),NaN,5.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN,5.0,4.0,NaN,4.0,5.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,2.5,4.5,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,3.5,4.0,4.5,3.0,NaN,NaN,NaN,3.0,5.0,NaN,NaN,4.0,NaN,4.5,4.5,NaN,NaN,NaN,3.0,NaN,NaN,3.5,NaN,NaN,4.0,5.0,3.0,4.0,4.0,NaN,NaN,3.0,NaN,5.0,2.0,NaN,3.0,NaN,NaN,5.0,4.0,NaN,3.5,NaN,NaN,NaN,NaN,NaN,4.0,NaN,4.0,NaN,NaN,NaN,3.5,3.0,4.0,NaN,NaN,NaN,3.5,NaN,5.0,5.0,NaN,4.0,3.0,4.0,4.0,NaN,4.0,NaN,NaN,NaN,NaN,4.0,3.5,NaN,NaN,2.0,NaN,4.0,NaN,NaN,NaN,5.0,NaN,NaN,3.0,NaN,3.5,5.0,3.5,3.5,NaN,5.0,NaN,NaN,3.5
Aladdin (1992),NaN,4.0,5.0,3.5,3.0,5.0,4.0,NaN,NaN,NaN,5.0,NaN,NaN,4.0,NaN,4.0,4.0,NaN,3.5,NaN,2.5,NaN,1.0,3.5,NaN,4.0,NaN,4.5,NaN,NaN,NaN,3.5,NaN,3.0,NaN,NaN,5.0,4.0,NaN,5.0,NaN,NaN,NaN,4.0,4.0,NaN,NaN,4.5,5.0,NaN,NaN,4.0,3.0,5.0,4.0,NaN,4.0,NaN,4.0,NaN,4.5,4.0,NaN,4.0,3.0,NaN,4.0,NaN,4.0,NaN,NaN,NaN,NaN,3.5,3.0,NaN,NaN,NaN,3.5,NaN,4.5,NaN,NaN,4.0,5.0,4.0,5.0,4.0,2.5,NaN,4.0,3.0,2.0,NaN,4.0,4.0,NaN,NaN,5.0,NaN,2.0,4.0,3.0,4.0,4.0,4.5,3.5,4.0,3.5,4.0,3.0,3.5,4.5,NaN,NaN,NaN,4.0,4.0,NaN,4.5,2.0,4.5,4.0,4.5,NaN,4.0,3.0,3.5,NaN,3.5,NaN,3.0,NaN


In [39]:
movie_pivot.shape

(205, 133)

In [40]:
movie_pivot.fillna(0, inplace=True)

In [42]:
movie_pivot.head()

userId,1,4,6,18,19,20,21,28,41,42,45,50,51,57,62,63,64,66,68,73,82,84,89,91,103,104,105,111,119,122,125,132,135,140,156,160,169,177,182,186,187,198,199,200,202,212,217,219,220,221,222,226,232,234,239,246,249,263,274,275,282,288,290,292,294,298,304,305,307,312,313,318,325,328,330,332,339,352,354,356,357,365,368,372,380,381,382,385,387,391,414,425,428,432,434,438,448,452,453,462,469,474,477,480,483,484,489,495,509,514,517,525,534,555,560,561,562,563,567,573,580,586,590,594,596,597,599,600,603,605,606,608,610
title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2001: A Space Odyssey (1968),0.0,0.0,0.0,4.0,3.0,0.0,0.0,3.0,2.0,5.0,4.0,4.5,5.0,5.0,0.0,3.5,1.0,0.0,1.5,4.0,0.0,0.0,0.0,3.5,4.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4.5,5.0,0.0,1.5,5.0,5.0,4.0,4.0,0.0,0.0,5.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.5,0.0,2.5,1.0,0.0,5.0,5.0,0.0,0.0,4.0,4.0,4.0,3.5,5.0,3.0,3.5,5.0,4.0,1.5,0.0,5.0,4.0,3.5,0.0,0.0,0.0,3.0,2.0,4.0,0.0,0.0,4.0,5.0,5.0,5.0,0.0,0.0,0.0,4.5,0.0,4.0,3.0,0.0,2.0,5.0,4.0,4.5,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,3.5,0.0,5.0,0.0,4.0,0.0,5.0,4.0,5.0,0.0,5.0,3.0,4.5
300 (2007),0.0,0.0,0.0,3.5,0.0,0.0,3.5,4.5,2.0,0.0,0.0,0.0,5.0,0.0,4.5,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.5,4.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,2.5,0.0,0.0,5.0,0.0,3.5,0.0,4.0,4.5,0.0,0.0,0.0,5.0,0.0,4.0,0.0,5.0,0.0,4.0,0.0,3.5,0.0,0.0,0.0,0.0,3.0,0.0,3.0,3.5,0.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,4.0,3.5,3.0,0.0,3.5,0.0,0.0,4.0,3.5,0.0,0.0,3.5,0.0,4.0,0.0,0.0,4.0,3.5,5.0,3.0,0.0,0.0,0.0,0.0,0.0,4.5,4.0,4.0,3.0,4.0,0.0,3.5,0.0,0.0,3.5,4.0,0.0,3.5,3.5,0.0,0.0,0.0,4.5,5.0,0.0,3.5,0.0,0.0,0.0,3.0,0.0,0.0,3.0,0.0,5.0,4.0
Ace Ventura: Pet Detective (1994),0.0,0.0,3.0,2.5,2.0,0.0,0.0,0.0,0.0,2.0,4.0,0.0,3.0,0.0,3.0,5.0,3.5,0.0,2.5,0.0,3.5,0.0,0.0,3.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,1.5,0.0,0.0,3.0,0.0,3.5,4.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,3.0,4.0,2.0,0.0,4.0,4.5,0.0,2.0,0.0,0.0,3.5,0.0,4.5,0.0,2.0,3.0,0.0,2.0,0.0,3.0,0.0,0.0,4.0,0.0,0.0,3.5,0.0,0.0,0.5,0.0,0.0,0.0,2.5,1.5,0.0,0.0,0.0,0.0,5.0,3.0,0.0,3.0,3.5,0.0,2.0,3.5,0.5,1.5,3.0,4.0,3.0,0.0,4.0,0.0,0.0,1.0,3.0,3.5,4.0,0.0,1.5,4.5,4.0,0.0,2.5,0.0,3.5,4.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,2.5,3.0,0.0,0.0,0.0,3.5,3.0
Airplane! (1980),0.0,5.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,5.0,4.0,0.0,4.0,5.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,2.5,4.5,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,3.5,4.0,4.5,3.0,0.0,0.0,0.0,3.0,5.0,0.0,0.0,4.0,0.0,4.5,4.5,0.0,0.0,0.0,3.0,0.0,0.0,3.5,0.0,0.0,4.0,5.0,3.0,4.0,4.0,0.0,0.0,3.0,0.0,5.0,2.0,0.0,3.0,0.0,0.0,5.0,4.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,0.0,3.5,3.0,4.0,0.0,0.0,0.0,3.5,0.0,5.0,5.0,0.0,4.0,3.0,4.0,4.0,0.0,4.0,0.0,0.0,0.0,0.0,4.0,3.5,0.0,0.0,2.0,0.0,4.0,0.0,0.0,0.0,5.0,0.0,0.0,3.0,0.0,3.5,5.0,3.5,3.5,0.0,5.0,0.0,0.0,3.5
Aladdin (1992),0.0,4.0,5.0,3.5,3.0,5.0,4.0,0.0,0.0,0.0,5.0,0.0,0.0,4.0,0.0,4.0,4.0,0.0,3.5,0.0,2.5,0.0,1.0,3.5,0.0,4.0,0.0,4.5,0.0,0.0,0.0,3.5,0.0,3.0,0.0,0.0,5.0,4.0,0.0,5.0,0.0,0.0,0.0,4.0,4.0,0.0,0.0,4.5,5.0,0.0,0.0,4.0,3.0,5.0,4.0,0.0,4.0,0.0,4.0,0.0,4.5,4.0,0.0,4.0,3.0,0.0,4.0,0.0,4.0,0.0,0.0,0.0,0.0,3.5,3.0,0.0,0.0,0.0,3.5,0.0,4.5,0.0,0.0,4.0,5.0,4.0,5.0,4.0,2.5,0.0,4.0,3.0,2.0,0.0,4.0,4.0,0.0,0.0,5.0,0.0,2.0,4.0,3.0,4.0,4.0,4.5,3.5,4.0,3.5,4.0,3.0,3.5,4.5,0.0,0.0,0.0,4.0,4.0,0.0,4.5,2.0,4.5,4.0,4.5,0.0,4.0,3.0,3.5,0.0,3.5,0.0,3.0,0.0


In [43]:
movie_sparse = csr_matrix(movie_pivot)

In [44]:
movie_sparse

<205x133 sparse matrix of type '<class 'numpy.float64'>'
	with 13839 stored elements in Compressed Sparse Row format>

In [45]:
model = NearestNeighbors(algorithm='brute')

In [46]:
model.fit(movie_sparse)

NearestNeighbors(algorithm='brute')

In [49]:
np.where(movie_pivot.index == 200)

(array([], dtype=int64),)

In [81]:
def based_content_filtering(movie_id):
    distances, suggestions = model.kneighbors(movie_pivot.iloc[movie_id, :].values.reshape(1, -1), n_neighbors=6)

    for i in range(len(suggestions[0])):
        if i == 0:
            print('The Suggestions For', movie_pivot.index[movie_id], "Are : ")
        else:
            # suggestions_movie.append(movie_pivot.index[suggestions[0][i]])
            print(movie_pivot.index[suggestions[0][i]])


In [63]:
id = int(input('Enter the movie id : '))

Enter the movie id : 50


In [64]:
movie_pivot.index[id]

'Dark Knight, The (2008)'

In [84]:
suggestions_movie = based_content_filtering(id)

The Suggestions For Dark Knight, The (2008) Are : 
Iron Man (2008)
Inception (2010)
WALL·E (2008)
Batman Begins (2005)
Up (2009)
